# Оптимизация инкассаций

## Загрузка данных и библиотек

In [1]:
import pandas as pd
from itertools import combinations

In [2]:
# загружаем данные по времени работы АТМ и стоимости инкассации
df_atm_info = pd.read_csv("atm_info.csv", sep = ';')
df_atm_info['worktime_split'] = df_atm_info['worktime_split'].apply(eval)

In [3]:
# загружаем данные по остаткам и внесениям - снятиям
df_in_out_train = pd.read_csv("test_private.csv")

In [4]:
# загружаем данные по ставке фондирования
funding_rate = pd.read_csv("funding_rate.csv", sep=';')

## Константы

In [5]:
DAYS_HORIZON = 8 # на сколько дней (не включая последний) делаем график инкассаций
# Проверенная корректная работа только при DAYS_HORIZON = 8

COLUMNS = list(df_in_out_train.columns)[:9] # названия столбцов таблицы ответа
COLUMNS.pop(1) # исключаем 'remains'

BASE_DAY = str((pd.to_datetime(COLUMNS[1]) - pd.to_timedelta('1 days')).date()) # нулевой день
LAST_DAY = COLUMNS[-1] # последний день

INF = 10**100 # очень большое число

MIN_REMAINS = 500_001 # минимальный допустимый остаток на конец дня
MAX_ENCASHING = 20_000_000 # максимальная допустимая сумма инкассации

## Функции

In [6]:
def get_weekday(date:str) -> int:
    '''
    Возвращает номер дня недели заданной даты
    
    Параметры
    ---------
    date : строка вида 'YYYY-MM-DD' с датой
    
    Возвращает
    ----------
    int - номер дня недели от 1(пн) до 7(вс)
    '''
    weekday = pd.to_datetime(date).isoweekday() # достаем день недели от 1(пн) до 7(вс)
    return weekday

In [7]:
def create_temp(row:pd.DataFrame) -> pd.DataFrame:
    '''
    Создаёт временную таблицу для банкомата.
    
    Параметры
    ---------
    row : pandas.DataFrame, содержащий 1 строку.
    Эта строка должна быть из датафрейма исходных
    данных `df_in_out_train`
    
    Возвращает
    ----------
    pandas.DataFrame, содержащий информацию о
    снятиях/внесениях, датах работы конкретного
    банкомата
    '''
    temp = row.T.reset_index() # транспонируем строку, сбрасываем индекс
    temp.columns = ['date', 'money_flow'] # первые 2 столбцца - дата и сумма снятий/внесений
    temp['atm_id'] = temp.loc[0, 'money_flow'] # столбец с id создаем и заполняем id АТМ
    temp['date'][1] = BASE_DAY # меняем `remains` на первую дату 
    temp = temp.head(DAYS_HORIZON + 1).tail(DAYS_HORIZON) # оствялем только нужные строки
    
    # рабочие дни
    days = list(*(df_atm_info[df_atm_info['atm_id'] == temp['atm_id'][1]]
                  ['worktime_split']))
    
    # получаем день недели базового дня
    weekday_slice_index = get_weekday(BASE_DAY) 
    if DAYS_HORIZON == 8:
        # "двигаем" график, так, чтобы он совпал с изучаемыми днями
        allowed_days = [0] + days[weekday_slice_index:] + days[:weekday_slice_index]
    elif DAYS_HORIZON == 11:
        allowed_days = [0] + days[weekday_slice_index:] + [days[:weekday_slice_index]] + [0] * 3
        
    # создаём столбец
    temp['allowed_days'] = allowed_days
    
    return temp

In [8]:
def get_combinations(temp:pd.DataFrame) -> dict:
    '''
    Даёт для конкретного банкомата
    комбинации возможных дней инкассации
    
    Параметры
    ---------
    temp : pandas.DataFrame, полученный
    функцией `create_temp`
    
    Возвращает
    ----------
    dict, содержащий
        ключи - количество дней (int),
        значения - возможные комбинации дней (tuple)
    '''
    total_days_encash_dates = {} # тут будут хранится пары 'количество дней':['даты']
    allowed_dates = temp[temp['allowed_days'] == 1]['date'] # выбираем только рабочие дни
    
    # получаем все возможные сочетания, кладём их в словарь
    for days_encash in range(allowed_dates.shape[0] + 1):
        total_days_encash_dates[days_encash] = list(
            combinations(list(allowed_dates), days_encash))
        
    return total_days_encash_dates

In [9]:
def get_encashing_amount(days_combination:list, temp:pd.DataFrame) -> pd.DataFrame:
    '''
    Исходя из дней и временной таблицы
    пополняет временную таблицу двумя столбцами:
    баланс наличности на конец дня в банкомате
    и объем инкассации в этот день
    
    Параметры
    ---------
    days_combination : list, комбинация дней инкассации
    
    temp : pandas.DataFrame, полученный
    функцией `create_temp`
    
    Возвращает
    ----------
    pandas.DataFrame, содержащий, помимо всех столбцов
    temp, столбцы 
        `encashing_amount` - объем инкассации
        в определённый день
        `balance` - остаток наличности на конец дня
    '''
    # замена None пустым списком (вариант отсутствия инкассации)
    if days_combination is None:
        days_combination = []
    
    # дни инкассации (без дубликатов)
    days_encash = sorted(list(set([BASE_DAY] + days_combination + [LAST_DAY])))
    
    # столбец с логическими значениями - "возможна ли инкассация в этот день"
    temp['encashing'] = temp['date'].apply(lambda x: x in days_combination)

    # снятия и внесения
    flow = list(temp['money_flow'])  
    
    # почти пустой баланс (= остаток), известен только на 1 день 
    balance = [flow[0]]  
    
    # дни когда можно инкассировать
    encashing = list(temp['encashing'])  
    
    # в этом списке будет находиться объем инкассации  
    encashing_amount = [0] * DAYS_HORIZON 
    
    # заполняем баланс с учётом трат
    for j in range(1, len(flow)):
        balance.append(balance[j - 1] + flow[j])

    # выбор текущего дня инкассации и следующего
    for i in range(len(encashing)):
        if encashing[i]:
            cur_day = i  # текущий день
            next_day = None  # день, до которого остаток должен быть >= 500k
            
            # определяем текущий день инкассации и следующий
            for j in range(i + 1, len(encashing)):
                if encashing[j]:
                    next_day = j
                    break
            if next_day is None:
                next_day = DAYS_HORIZON # если следущего нет - принимаем его за последний

            # определяем минимальную сумму снятий и внесений на
            # отрезке от текущего дня до следущего
            # только для тех случаев, когда эта сумма < MIN_REMAINS
            minimum = MIN_REMAINS-1
            flow_sum = 0
            for money in flow[cur_day:next_day]: # итерируемся от текущего до следующего
                flow_sum += money # сумма внесений
                if flow_sum < minimum: # если меньше чем минимум (на первом шаге MIN_REMAINS-1)
                    minimum = flow_sum # обновляем минимум

            cur_amount = MIN_REMAINS - minimum  # деньги которые необходимо внести
            if cur_amount > 0: # проверка, чтобы не внести отрицательную сумму
                encashing_amount[i] = cur_amount # заполняем список

    # считаем баланс (остатки)
    for i in range(1, len(balance)):
        if encashing_amount[i] > 0:
            balance[i] = encashing_amount[i] + flow[i]
        else:
            balance[i] = balance[i - 1] + flow[i]
            
    # добавляем их в таблицу столбцами
    temp['encashing_amount'] = encashing_amount
    temp['balance'] = balance
    
    return temp

In [10]:
def checker(changed_temp) -> bool:
    '''
    Проверяет, проходит ли таблица
    по заданным ограничениям
    
    Параметры
    ---------
    changed_temp : pandas.DataFrame,
    полученный функцией `get_encashing_amount`
    
    Возвращает
    ----------
    bool - логическое значение, а именно:
        True - если не нарушает ограничения
        False - если нарушает
    '''
    # убираем первую строку (с нулевым днём)
    changed_temp = changed_temp[1:]
    
    # два обязательных условия 
    cond1 = changed_temp['balance'].min() > MIN_REMAINS-1
    cond2 = changed_temp['encashing_amount'].max() < MAX_ENCASHING
    return (cond1 and cond2)

In [11]:
def get_costs(changed_temp:pd.DataFrame) -> float:
    '''
    Для предложенного варианта инкассации
    вычисляет все траты, а именно:
    сумма денег потраченных на инкассацию за неделю
    плюс сумма денег которые "сжигает" инфляция
    
    Параметры
    ---------
    changed_temp : pandas.DataFrame,
    полученный функцией `get_encashing_amount`
    
    Возвращает
    ----------
    float - траты
    '''
    costs = 0
    atm_id = changed_temp['atm_id'][1] # запоминаем id банкомата
    
    # находим стоимость инкассации этого банкомата
    encashing_cost = df_atm_info[df_atm_info['atm_id'] ==
                           atm_id]['incasationcost'].values[0] 
    # и прибавляем их к костам
    costs += (changed_temp['encashing_amount'] > 0).sum() * encashing_cost
    
    # джоин таблицы с ставкой рефинансирования по дате
    changed_temp_funding = changed_temp.merge(funding_rate,
                                              left_on='date',
                                              right_on='value_day')
    # находим траты связанные с инфляцией
    inflation = (changed_temp_funding['balance'] * changed_temp_funding['funding_rate']/365).sum()
    # и прибавляем их к костам
    costs += inflation
    return costs

In [12]:
def get_best_days(temp:pd.DataFrame) -> list:
    '''
    Для временной таблицы, не содержащей
    столбцов баланса и объема инкассации,
    перебирает все возможные комбинации дней.
    
    Выбирает лучшую комбинацию исходя из
    минимизации трат
    
    Параметры
    ---------
    temp : pandas.DataFrame,
    полученный функцией `create_temp`
    
    Возвращает
    ----------
    list - список дней инкассации
    '''
    # получаем комбинации
    combs = get_combinations(temp)
    # до цикла обозначим минимальные косты очень большим числом
    min_cost = INF
    # если вдруг условие ни разу не выполнится, вернем None
    days_comb = None

    # итерируемся по ключам и значениям словаря
    for total_days, dates in combs.items():
        for date in dates: # значение - список кортежей с датами, итерируемся по ним
            days_combination = list(date) 
            # получаем измененную временную таблицу
            changed_temp = get_encashing_amount(days_combination, temp)

            if checker(changed_temp): # если выполняются ограничения
                # запоминаем косты
                costs = get_costs(changed_temp)

                if costs < min_cost: # если они оказались меньше минимальных найденных
                    min_cost = costs # меняем минмиум
                    days_comb = days_combination # и запоминаем даты
    return days_comb

In [13]:
def get_answer(changed_temp:pd.DataFrame) -> pd.DataFrame:
    '''
    Формирует строку ответа
    
    Параметры
    ---------
    changed_temp : pandas.DataFrame,
    полученный функцией `get_encashing_amount`
    
    Возвращает
    ----------
    pandas.DataFrame - строка в формате ответа
    '''
    # преобразуем таблицу в нужный вид
    # а именно - берем столбец с суммой инкассации
    # и транспонируем его
    ans = changed_temp[['date',
               'encashing_amount']].head(DAYS_HORIZON).T.reset_index().drop(1, axis=1)
    # меняем значение в первой ячейке на id банкомата
    ans['index'] = changed_temp['atm_id'][1]
    # переименовывем столбцы
    ans.columns = COLUMNS
    # сохраняем только нужную нам строку
    answer = ans.tail(1)
    return answer

In [14]:
def sub_main(row:pd.DataFrame) -> pd.DataFrame:
    '''
    Принимает строку, возвращает
    строку-ответ
    
    Параметры
    ---------
    row : pandas.DataFrame, содержащий 1 строку.
    Эта строка должна быть из датафрейма исходных
    данных `df_in_out_train`
    
    Возвращает
    ----------
    pandas.DataFrame - строка в формате ответа
    '''
    temp = create_temp(row) # создаем временную таблицу
    best_days = get_best_days(temp) # получаем лучшие дни
    return get_answer(get_encashing_amount(best_days, temp)) # возвращаем строку ответа

In [15]:
def main(generator=df_in_out_train.index):
    '''
    Для всех банкоматов формирует строки
    ответов и соединяет их в таблицу
    
    Параметры
    ---------
    generator : генератор значений
    по умолчанию - индексы датафрейма исходных
    данных
    
    Возвращает
    ----------
    pandas.DataFrame - таблица в формате ответа
    '''
    # создаем пустой датафрейм
    final_answer = pd.DataFrame(columns=COLUMNS)
    for row_number in generator: 
        final_answer = pd.concat([
            final_answer,
            sub_main(pd.DataFrame(df_in_out_train.iloc[row_number]).T)
        ]) # добавляем на каждом шаге строку ответа
    return final_answer

## Решение

In [16]:
answer_sample = main(generator=range(1))
answer_sample # семпл ответа

,atm_id,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07
1,32a0a3467bc2255eea631b4411d0db92,0.0,0.0,0.0,5071601.0,0.0,7713001.0,0.0


In [17]:
# answer = main()
# answer.to_csv('solution.csv', index=None) # полный ответ